## Problem Set 3
_MaCSS 222 Applied Statistics II Spring 2025_   
_Bryan Graham_   
_University of California at Berkeley_      
_March 2025_   
<br>
<br>
In this problem set you will replicate key components of the analysis reported in the paper by Patrick McEwan, Erin Murphy-Graham, and collaborators, "Improving middle school quality in poor countries: evidence from the Honduras Sistema de Aprendizaje Tutorial" (_Educational Evaluation and Policy Analysis_ 37, 1 (2015), pp. 113 - 137). This paper studies the efficacy of an alternative rural secondary education program, _Sistema de Aprendizaje Tutorial_ (SAT), in five departments in Honduras. If you have not done so already, please read this article.

**Code citation:**

<br>
Graham, Bryan S. (2025). "Replication of Sistema de Aprendizaje Tutorial Evaluation," (Version 1.0) [Computer program]. Available at http://bryangraham.github.io/econometrics/ (Accessed 27 May 2025)
<br>
<br>

In [34]:
# Direct Python to plot all figures inline (i.e., not in a separate window)
%matplotlib inline

# Load additional libraries
import sympy
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm

The replication dataset is in Stata format. Fortunately _pandas_ is able to read data in this format directly.

In [35]:
SAT_dir = '/Users/chris/Library/Mobile Documents/com~apple~CloudDocs/Berkeley/COMPSS 222 Applied Statistics II/Data/'

# Read in data
sat=pd.read_stata(SAT_dir+'sat_eepa_rep.dta')
sat.columns

Index(['n_escuela', 's08_work', 's08_workhome', 's08_repeat', 's09_attend',
       's09_work', 's09_workhome', 's10_attend', 's10_work', 's10_workhome',
       's_age', 'h08_motherschool', 'h08_fatherschool', 'h08_householdsize',
       'h08_usa', 'h_dirtfloor', 'h_rooms', 'h_water', 'h_sewer',
       'h_electricityhh', 's_female', 'h_distance1', 'h_distance2',
       'm_id_pairs', 'm_feeder', 'm_closed2009', 'm_closed2010', 'lenca',
       'c09_composite', 'c10_composite', 'D2', 'D3', 'D4', 'D5',
       'weight_test_c09_zmath', 'weight_test_c09_zlang',
       'weight_test_c10_zmath', 'weight_test_c10_zlang', 'c08_zlang',
       'c09_zlang', 'c10_zlang', 'c08_zmath', 'c09_zmath', 'c10_zmath',
       's_goods', 'c08_zmath_missing', 'c08_zlang_missing', 's08_work_missing',
       's08_workhome_missing', 's_female_missing', 's_age_missing',
       'lenca_missing', 's08_repeat_missing', 'h08_householdsize_missing',
       'h08_motherschool_missing', 'h08_fatherschool_missing',
       'h08_

In this problem set we will focus on assessed youth performance on Mathematics and Spanish Language tests. Tests were administered to all surveyed respondents at baseline as well as to respondents at follow-up in 2009 and 2010 (with various complications described in the paper). The dataset consists of $1,426$ youth residing in one of $47$ "SAT villages or one of $47$ "CEB villages". CEB stands for Centro de Educacion Basicos (CEB). This is the "default" rural secondary school option (when one is available). The $47$ (SAT), $47$ (CEB) combination is by design. These 94 schools constitue 47 matched pairs as described in the paper.

In [36]:
print(sat[['n_escuela', 'm_feeder', 'm_id_pairs', 'c08_zlang', 'c08_zlang_missing', 'c08_zmath', 'c08_zmath_missing']].describe())

# create bin dummies for "quintiles" of test score distributions
quantile_cuts = [0.00, 0.10, 0.20, 0.30, 0.40, 0.50, \
                 0.60, 0.70, 0.80, 0.90, 1.00]

for subject in ['math','lang']:
    for c in range(1,len(quantile_cuts)):
        # Equals 1 if in cth quantile of the reference standard normal distribution and zero otherwise
        sat['c08_z' + subject + '_qc' + str(c)] = \
                        ((sat['c08_z' + subject] >= sp.stats.norm.ppf(quantile_cuts[c-1])) & \
                         (sat['c08_z' + subject] <  sp.stats.norm.ppf(quantile_cuts[c]))).astype(int)
        # Make sure missing test scores get quantile dummies coded as missing as well    
        sat.loc[(sat['c08_z' + subject + '_missing']==1),'c08_z' + subject + '_qc' + str(c)] = np.nan    
            
print("")
print("List if unique feeder school codes:")
print(sorted(sat['n_escuela'].unique()))
print("")
print("List of unique matched SAT/CEB feeder pairs:")
print(sorted(sat['m_id_pairs'].unique()))
print("")
print("Treatment indicator codes:")
print(sat['m_feeder'].unique())

         n_escuela   m_id_pairs     c08_zlang  c08_zlang_missing    c08_zmath  \
count  1426.000000  1426.000000  1.426000e+03        1426.000000  1426.000000   
mean     54.555400    33.424264 -5.350207e-09           0.140252     0.000000   
std      33.806343    17.042945  9.271727e-01           0.347371     0.910369   
min       1.000000     2.000000 -2.668711e+00           0.000000    -3.327293   
25%      24.000000    18.000000 -5.902593e-01           0.000000    -0.593881   
50%      51.000000    35.000000  0.000000e+00           0.000000     0.000000   
75%      83.000000    49.000000  5.354011e-01           0.000000     0.539350   
max     118.000000    59.000000  3.111088e+00           1.000000     3.207138   

       c08_zmath_missing  
count        1426.000000  
mean            0.171108  
std             0.376735  
min             0.000000  
25%             0.000000  
50%             0.000000  
75%             0.000000  
max             1.000000  

List if unique feeder scho

In [37]:
# Add a treatment dummy & constant to dataset
sat['sat']      = (sat.m_feeder != 'C.E.B.').astype('int')
sat['constant'] = 1

# Form dummies for included matched SAT/CEB pairs
mp_dums = pd.get_dummies(sat['m_id_pairs'].astype('category'), prefix='mp')

# Get list of indices for all matched village pairs
mp_list = sorted(sat['m_id_pairs'].unique())

# Get dummy variable names for all matched village pairs (except last pair; avoids dummy variable trap)
mp_vnames = mp_dums.loc[:,'mp_' + str(mp_list[0]) :'mp_'+ str(mp_list[-2])].columns

# Concatenate matched pair dummies to dataframe
sat = pd.concat([sat, mp_dums], axis=1)

In [38]:
Baseline    = ['c08_zmath','c08_zlang']    
Work        = ['s08_work', 's08_workhome']
Student     = ['s_female', 's_age', 'lenca', 's08_repeat'] 
Family      = ['h08_householdsize', 'h08_motherschool', 'h08_fatherschool', 'h08_usa', \
               's_goods', 'h_rooms', 'h_dirtfloor', 'h_water', 'h_sewer', 'h_electricityhh']  
Access      = ['h_distance1', 'h_distance2']
Departments = ['D2', 'D3', 'D4', 'D5']
Missing     = ['c08_zmath_missing', 'c08_zlang_missing', 's08_work_missing', \
               's08_workhome_missing', 's_female_missing', 's_age_missing', \
               'lenca_missing', 's08_repeat_missing', 'h08_householdsize_missing', \
               'h08_motherschool_missing', 'h08_fatherschool_missing', \
               'h08_usa_missing', 's_goods_missing', 'h_rooms_missing', \
               'h_dirtfloor_missing', 'h_water_missing', 'h_sewer_missing', \
               'h_electricityhh_missing', 'h_distance1_missing', 'h_distance2_missing', \
               'D2_missing', 'D3_missing', 'D4_missing', 'D5_missing']

In [39]:
print("Variable Name              Mean      StdDev          n                SAT       CEB           Balance Reg.")
print("----------------------------------------------------------------------------------------------------------")
for var in Baseline+Work+Student+Family+Access:
    mean = sat.loc[sat[var+'_missing']!=1,var].mean()            # Mean across respondents with non-missing values
    std  = sat.loc[sat[var+'_missing']!=1,var].std()             # Standard deviation across respondents with non-missing values
    n    = sat.loc[sat[var+'_missing']!=1,var].count()           # Total number of respondents with non-missing values

    # Use groupby method to compute means separately across CEB and SAT villages
    ceb_mean, sat_mean = sat.loc[sat[var+'_missing']!=1,[var,'sat']].groupby('sat').mean().T.values[0,:]

    # Compute OLS fit of current var onto the SAT indicator as well as the matched pair dummies
    balance_test = sm.ols(formula = var + ' ~ sat + ' + ' + '.join(mp_vnames), data=sat)
    results = balance_test.fit(cov_type='cluster', cov_kwds={'groups': sat['n_escuela']})
    sat_coef = results.params['sat']
    sat_coef_se = results.bse['sat']
    
    # Print summary statistics with nice formating
    print(var.ljust(25) + "%10.6f" % mean + \
                       " (" + "%8.6f" % std + ")" + "     n = " + "%4.0f" % n + \
                       "%15.4f" % sat_mean + "  " + "%8.4f" % ceb_mean + \
                       "%15.6f" % sat_coef + " (" + "%8.6f" % sat_coef_se + ")" + \
                       ['','*'][1*(np.abs(sat_coef/sat_coef_se) >= sp.stats.norm.ppf(0.975))]) 

Variable Name              Mean      StdDev          n                SAT       CEB           Balance Reg.
----------------------------------------------------------------------------------------------------------
c08_zmath                 -0.000000 (1.000000)     n = 1182        -0.0225    0.0182      -0.022642 (0.080744)
c08_zlang                  0.000000 (1.000000)     n = 1226        -0.0143    0.0120      -0.015593 (0.070167)
s08_work                   0.397293 (0.489533)     n = 1256         0.3731    0.4180      -0.035803 (0.035374)
s08_workhome               0.851675 (0.355564)     n = 1254         0.8569    0.8472      -0.014937 (0.039598)
s_female                   0.518233 (0.499843)     n = 1426         0.5132    0.5228      -0.023010 (0.020094)
s_age                     12.973985 (1.438130)     n = 1403        13.0237   12.9286       0.103191 (0.108172)
lenca                      0.411628 (0.492319)     n = 1290         0.2930    0.5184      -0.220238 (0.052866)*
s08_repe

## 2.2 Reproducing Table 1

In [40]:
import statsmodels.formula.api as smf  # Corrected import
from scipy.stats import norm

# Define Variable Labels
var_labels = {
    "c08_zmath": "Mathematics test score",
    "c08_zlang": "Language test score",
    "s08_work": "1 = salaried or unsalaried work outside home last week; 0 = not",
    "s08_workhome": "1 = unsalaried worked in home last week on chores; 0 = not",
    "s_female": "1 = female; 0 = male",
    "s_age": "Age on October 15, 2008",
    "lenca": "1 = identifies as Lenca; 0 = not",
    "s08_repeat": "1 = repeated ≥1 grade; 0 = not",
    "h08_householdsize": "Number of household members",
    "h08_motherschool": "Mother’s years of schooling",
    "h08_fatherschool": "Father’s years of schooling",
    "h08_usa": "1 = ≥1 relative living in United States; 0 = not",
    "s_goods": "Index of 10 household assets",
    "h_rooms": "Number of rooms in dwelling",
    "h_dirtfloor": "1 = dirt floor; 0 = not",
    "h_water": "1 = piped water in dwelling; 0 = not",
    "h_sewer": "1 = sewer/septic connection; 0 = not",
    "h_electricityhh": "1 = electricity connection; 0 = not",
    "h_distance1": "Kilometers to nearest SAT or CEB",
    "h_distance2": "Kilometers to nearest Instituto",
    "m_closed2009": "1 = attritor in October 2009; 0 = not",
    "m_closed2010": "1 = attritor in October 2010; 0 = not",
    "c09_composite": "2008 composite score, 2009 attritors",
    "c10_composite": "2008 composite score, 2010 attritors"
}

panelA_vars = [
    "c08_zmath", "c08_zlang", "s08_work", "s08_workhome", "s_female", 
    "s_age", "lenca", "s08_repeat", "h08_householdsize", "h08_motherschool", 
    "h08_fatherschool", "h08_usa", "s_goods", "h_rooms", "h_dirtfloor", 
    "h_water", "h_sewer", "h_electricityhh", "h_distance1", "h_distance2"
]
panelB_vars = ["m_closed2009", "m_closed2010"]
panelC_vars = ["c09_composite", "c10_composite"]

# Combine all panels for structured iteration
panels = {
    "Panel A: Baseline student and household characteristics": panelA_vars,
    "Panel B: Attrition indicators": panelB_vars,
    "Panel C: Composite test scores (October 2008) among attritors in October 2009 and 2010": panelC_vars
}

# Iterate through each panel and print results
for panel_title, variables in panels.items():
    print(f"\n{panel_title}")
    print("-" * 110)

    for var in variables:
        if var not in sat.columns:
            continue  # Skip missing variables

        # Get correct variable label
        var_label = var_labels.get(var, var)

        # Handle missing values properly
        missing_col = var + '_missing'
        valid_rows = sat.loc[sat[missing_col] != 1, var] if missing_col in sat.columns else sat[var].dropna()

        # Compute summary statistics
        mean = valid_rows.mean()
        std = valid_rows.std()
        n = valid_rows.count()

        # Compute means for SAT and CEB groups
        group_means = valid_rows.groupby(sat.loc[valid_rows.index, 'sat']).mean()
        ceb_mean, sat_mean = (group_means[0], group_means[1]) if 0 in group_means.index and 1 in group_means.index else (np.nan, np.nan)

        # Drop missing values for OLS regression
        sat_ols = sat.dropna(subset=[var, 'sat'] + list(mp_vnames))

        # Run regression only if enough data exists
        if len(sat_ols) > 10:  
            balance_test = smf.ols(f"{var} ~ sat + {' + '.join(mp_vnames)}", data=sat_ols).fit(cov_type='cluster', cov_kwds={'groups': sat_ols['n_escuela']})
            sat_coef = balance_test.params.get('sat', np.nan)
            sat_coef_se = balance_test.bse.get('sat', np.nan)
        else:
            sat_coef, sat_coef_se = np.nan, np.nan

        # Compute significance
        t_stat = np.abs(sat_coef / sat_coef_se) if sat_coef_se > 0 else np.nan
        significance = "***" if t_stat >= norm.ppf(0.99) else "**" if t_stat >= norm.ppf(0.975) else "*" if t_stat >= norm.ppf(0.90) else ""

        # Print formatted table row
        print(var_label.ljust(55) + 
              f"{mean:10.6f} ({std:8.6f})    n = {n:4}   " +
              f"{sat_mean:10.4f}  {ceb_mean:8.4f}  " +
              f"{sat_coef:15.6f} ({sat_coef_se:8.6f}){significance}")



Panel A: Baseline student and household characteristics
--------------------------------------------------------------------------------------------------------------
Mathematics test score                                  -0.000000 (1.000000)    n = 1182      -0.0225    0.0182        -0.022642 (0.080744)
Language test score                                      0.000000 (1.000000)    n = 1226      -0.0143    0.0120        -0.015593 (0.070167)
1 = salaried or unsalaried work outside home last week; 0 = not  0.397293 (0.489533)    n = 1256       0.3731    0.4180        -0.035803 (0.035374)
1 = unsalaried worked in home last week on chores; 0 = not  0.851675 (0.355564)    n = 1254       0.8569    0.8472        -0.014937 (0.039598)
1 = female; 0 = male                                     0.518233 (0.499843)    n = 1426       0.5132    0.5228        -0.023010 (0.020094)
Age on October 15, 2008                                 12.973985 (1.438130)    n = 1403      13.0237   12.9286         0

## 2.3 Reproducing Table 3

This next block of code produces Columns 1 and 4 of Panels D and E of Table 3 in the paper. By uncommenting some code you can modify to also reproduce Columns 2 and 5 of these panels.

### Columns 1 and 4

In [46]:
for subject in ['math','lang']:
    for year in ['09','10']:
        
        # Find units with needed outcome variable
        cc_filter = sat['c' + year + '_z' + subject].notnull()

        # Find subset of matched pairs with needed outcome variable for
        # some youth in both SAT and CEB feeders
        cc_sample = sat[cc_filter].groupby('m_id_pairs'). \
                                   filter(lambda x: ((np.mean(x['sat']) > 0) & (np.mean(x['sat']) < 1)))

        
        # Calculate some basic statistics and initial non-response and attrition
        included_pairs = sorted(cc_sample['m_id_pairs'].unique()) # List of included matched pairs    
        target_filter  = sat['m_id_pairs'].isin(included_pairs)   # All target respondents in included pairs
        n_targ         = target_filter.sum()                      # Number of target respondents in included pairs
        base_filter    = target_filter & (sat['c08_zlang_missing']==0) & \
                                         (sat['c08_zmath_missing']==0)  # Target respondents
                                                                        # (fully) tested at baseline
                                                                        # in included pairs
        n_base         = base_filter.sum()                        # Number of target respondents with baseline info in
                                                                  # included pairs
       
        # Find subset of missingness indicators that are linearly independent
        echelon, indices = sympy.Matrix(cc_sample[Missing].values).rref()
        Missing_lin_ind = [Missing[i] for i in range(len(Missing)) if i in indices]
        
        # Weight to account for "under-testing" of at home youth during follow-up
        test_wgt = cc_sample['weight_test_c' + year + '_z' + subject]  
        
        print("")    
        print("Target respondents included matched pairs, n = "  + "%0.0f" % np.sum(n_targ))
        print( "...with baseline test scores,              n = "  + "%0.0f" % np.sum(n_base))
        print( "...with " + year + " " + subject + " test score,                n = " + "%0.0f" % len(cc_sample['m_id_pairs']))
        print("Sum of test_wgt (effective sample size),   n = " + "%0.0f" % np.sum(test_wgt))

        print("")    
        print("Number of matched pairs with (some) baseline language/math test scores, and follow-up")
        print("test scores, in both CEB and SAT feeders")
        print(len(included_pairs))
        print("")
        print("List of matched feeder pairs with enough complete cases")
        print(included_pairs)
        print("")

        
        # Formula
        formula = 'c' + year + '_z' + subject + ' ~ sat + ' + ' + '.join(mp_vnames)
        
        # Compute OLS fit of outcome onto the SAT indicator as well as the matched pair dummies
        att_estimate = sm.wls(formula, data=cc_sample, weights=test_wgt)
        results = att_estimate.fit(cov_type='cluster', cov_kwds={'groups': cc_sample['n_escuela']})
        print(results.summary())


Target respondents included matched pairs, n = 1426
...with baseline test scores,              n = 1182
...with 09 math test score,                n = 999
Sum of test_wgt (effective sample size),   n = 1254

Number of matched pairs with (some) baseline language/math test scores, and follow-up
test scores, in both CEB and SAT feeders
47

List of matched feeder pairs with enough complete cases
[2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 27, 28, 29, 30, 31, 33, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 59]

                            WLS Regression Results                            
Dep. Variable:              c09_zmath   R-squared:                       0.142
Model:                            WLS   Adj. R-squared:                  0.099
Method:                 Least Squares   F-statistic:                     360.1
Date:                Sat, 22 Mar 2025   Prob (F-statistic):           4.75e-88
Time:                    

/usr/local/lib/python3.9/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 47, but rank is 46
  warnings.warn('covariance of constraints does not have full '



Target respondents included matched pairs, n = 1426
...with baseline test scores,              n = 1182
...with 09 lang test score,                n = 1025
Sum of test_wgt (effective sample size),   n = 1279

Number of matched pairs with (some) baseline language/math test scores, and follow-up
test scores, in both CEB and SAT feeders
47

List of matched feeder pairs with enough complete cases
[2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 27, 28, 29, 30, 31, 33, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 59]

                            WLS Regression Results                            
Dep. Variable:              c09_zlang   R-squared:                       0.191
Model:                            WLS   Adj. R-squared:                  0.152
Method:                 Least Squares   F-statistic:                     572.5
Date:                Sat, 22 Mar 2025   Prob (F-statistic):           2.35e-97
Time:                   

### Columns 2 and 5

In [47]:
for subject in ['math','lang']:
    for year in ['09','10']:
        
        # Find units with needed outcome variable
        cc_filter = sat['c' + year + '_z' + subject].notnull()

        # Find subset of matched pairs with needed outcome variable for
        # some youth in both SAT and CEB feeders
        cc_sample = sat[cc_filter].groupby('m_id_pairs'). \
                                   filter(lambda x: ((np.mean(x['sat']) > 0) & (np.mean(x['sat']) < 1)))

        
        # Calculate some basic statistics and initial non-response and attrition
        included_pairs = sorted(cc_sample['m_id_pairs'].unique()) # List of included matched pairs    
        target_filter  = sat['m_id_pairs'].isin(included_pairs)   # All target respondents in included pairs
        n_targ         = target_filter.sum()                      # Number of target respondents in included pairs
        base_filter    = target_filter & (sat['c08_zlang_missing']==0) & \
                                         (sat['c08_zmath_missing']==0)  # Target respondents
                                                                        # (fully) tested at baseline
                                                                        # in included pairs
        n_base         = base_filter.sum()                        # Number of target respondents with baseline info in
                                                                  # included pairs
       
        # Find subset of missingness indicators that are linearly independent
        echelon, indices = sympy.Matrix(cc_sample[Missing].values).rref()
        Missing_lin_ind = [Missing[i] for i in range(len(Missing)) if i in indices]
        
        # Weight to account for "under-testing" of at home youth during follow-up
        test_wgt = cc_sample['weight_test_c' + year + '_z' + subject]  
        
        print("")    
        print("Target respondents included matched pairs, n = "  + "%0.0f" % np.sum(n_targ))
        print( "...with baseline test scores,              n = "  + "%0.0f" % np.sum(n_base))
        print( "...with " + year + " " + subject + " test score,                n = " + "%0.0f" % len(cc_sample['m_id_pairs']))
        print("Sum of test_wgt (effective sample size),   n = " + "%0.0f" % np.sum(test_wgt))

        print("")    
        print("Number of matched pairs with (some) baseline language/math test scores, and follow-up")
        print("test scores, in both CEB and SAT feeders")
        print(len(included_pairs))
        print("")
        print("List of matched feeder pairs with enough complete cases")
        print(included_pairs)
        print("")

        
        # Formula
        formula = 'c' + year + '_z' + subject + ' ~ sat + ' + ' + '.join(Baseline+Work+Student+Family+Access+Departments+Missing_lin_ind) + ' + ' + ' + '.join(mp_vnames) 
        
        # Compute OLS fit of outcome onto the SAT indicator as well as the matched pair dummies
        att_estimate = sm.wls(formula, data=cc_sample, weights=test_wgt)
        results = att_estimate.fit(cov_type='cluster', cov_kwds={'groups': cc_sample['n_escuela']})
        print(results.summary())


Target respondents included matched pairs, n = 1426
...with baseline test scores,              n = 1182
...with 09 math test score,                n = 999
Sum of test_wgt (effective sample size),   n = 1254

Number of matched pairs with (some) baseline language/math test scores, and follow-up
test scores, in both CEB and SAT feeders
47

List of matched feeder pairs with enough complete cases
[2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 27, 28, 29, 30, 31, 33, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 59]

                            WLS Regression Results                            
Dep. Variable:              c09_zmath   R-squared:                       0.430
Model:                            WLS   Adj. R-squared:                  0.378
Method:                 Least Squares   F-statistic:                     1241.
Date:                Sat, 22 Mar 2025   Prob (F-statistic):          2.31e-117
Time:                    

/usr/local/lib/python3.9/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 85, but rank is 84
  warnings.warn('covariance of constraints does not have full '



Target respondents included matched pairs, n = 1426
...with baseline test scores,              n = 1182
...with 09 lang test score,                n = 1025
Sum of test_wgt (effective sample size),   n = 1279

Number of matched pairs with (some) baseline language/math test scores, and follow-up
test scores, in both CEB and SAT feeders
47

List of matched feeder pairs with enough complete cases
[2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 27, 28, 29, 30, 31, 33, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 59]

                            WLS Regression Results                            
Dep. Variable:              c09_zlang   R-squared:                       0.525
Model:                            WLS   Adj. R-squared:                  0.482
Method:                 Least Squares   F-statistic:                 3.884e+04
Date:                Sat, 22 Mar 2025   Prob (F-statistic):          5.90e-187
Time:                   